In [2]:
from typing import TYPE_CHECKING
from transformers.utils import _LazyModule
from transformers.utils.import_utils import define_import_structure
if TYPE_CHECKING:  # 静态检查工具检测时
    from transformers.models.gpt2.configuration_gpt2 import *
    from transformers.models.gpt2.modeling_flax_gpt2 import *
    from transformers.models.gpt2.modeling_gpt2 import *
    from transformers.models.gpt2.modeling_tf_gpt2 import *
    from transformers.models.gpt2.tokenization_gpt2 import *
    from transformers.models.gpt2.tokenization_gpt2_fast import *
    from transformers.models.gpt2.tokenization_gpt2_tf import *
else: # 否则,懒加载
    import sys
    # _file = globals()["__file__"]
    # sys.modules[__name__] = _LazyModule(__name__, _file, define_import_structure(_file), module_spec=__spec__)

In [4]:
from collections import OrderedDict
from typing import Any, List, Mapping, Optional
from transformers import PreTrainedTokenizer, TensorType, is_torch_available
from transformers.configuration_utils import PretrainedConfig
from transformers.onnx import OnnxConfigWithPast, PatchingSpec
from transformers.utils import logging

In [5]:
logger = logging.get_logger(__name__)

In [ ]:
# 场景	会参考 keys_to_ignore_at_inference 吗？	说明
# model(**inputs)	❌ 不会参考	直接返回所有输出，包括 past_key_values
# model.generate(...)	✅ 会参考	自动剔除非必要字段，生成结果中可能会排除这些键
# transformers.pipeline(...)	✅ 会参考	自动过滤掉这些键，保证输出简洁
# Trainer.predict(...)	✅ 会参考	只返回用户关心的主输出（如 logits、labels）
# 这两个设置作用时间点不同：
# 设置	生效时间点	控制什么
# use_cache	forward 前	是否生成缓存
# keys_to_ignore_at_inference	forward 后	推理时是否过滤掉缓存字段
# 如果你是自己调用 model(input_ids)，不会有任何影响，past_key_values 总会返回（除非你手动 use_cache=False）
# ；但如果你用的是 .generate() 或 pipeline，则可能自动帮你裁剪掉这些键。

In [7]:
# 默认的GPT2Config 返回openai-community/gpt2相似的配置
class GPT2Config(PretrainedConfig):
    """
    ```python
    >>> from transformers import GPT2Config, GPT2Model

    >>> # Initializing a GPT2 configuration
    >>> configuration = GPT2Config()

    >>> # Initializing a model (with random weights) from the configuration
    >>> model = GPT2Model(configuration)

    >>> # Accessing the model configuration
    >>> configuration = model.config
    ```"""
    model_type = "gpt2"   # 指定模型类型
    # 模型在推理输出中可能会包含 past_key_values（即用于加速的缓存）。
    # 但对于某些任务（比如只获取最终 logits），这个缓存在后处理时可以忽略。
    # 并不是“加载配置时忽略”，而是“模型 forward 输出时，这些 key 可以选择性忽略”。
    # 是否输出缓存，取决于 use_cache 参数；是否忽略缓存，取决于 keys_to_ignore_at_inference。两者作用不同。
    # 模型在 forward 时依然会输出 past_key_values，只是告诉某些 API（如管道、Trainer 或序列生成等）在后处理或使用时可以忽略它。
    keys_to_ignore_at_inference = ["past_key_values"] 
    # 属性映射
    # 如果某些用户或框架代码使用标准化名称（如 hidden_size）来访问或设置配置，内部会自动映射到对应的实际字段（如 n_embd）。
    # 这在如下两种情况中会用到：
    # 用户调用 config.hidden_size，实际上访问的是 config.n_embd。
    # 使用 from_pretrained 加载配置时传入了 hidden_size=1024，会自动设置 n_embd=1024。
    # 这保证了兼容性与通用接口，即使底层实现字段名是非通用的（如 n_embd），也可以使用标准名字访问
    attribute_map = {
        "hidden_size": "n_embd",
        "max_position_embeddings": "n_positions",
        "num_attention_heads": "n_head",
        "num_hidden_layers": "n_layer",
    }

    def __init__(
        self,
        vocab_size=50257, # 词汇表大小
        n_positions=1024,  # 支持的最大序列长度
        n_embd=768, # 模型隐藏维度
        n_layer=12,  # Transformer 层数
        n_head=12,  # 多头注意力中 head 的数量
        n_inner=None, # 前馈网络的中间层维度（默认为 None，表示使用 n_embd）
        activation_function="gelu_new", # 前馈网络中使用的激活函数
        resid_pdrop=0.1, # 残差连接的 dropout 概率
        embd_pdrop=0.1, # 嵌入层的 dropout 概率
        attn_pdrop=0.1,  # 注意力权重的 dropout 概率
        layer_norm_epsilon=1e-5,  # LayerNorm 中的 epsilon，避免除以 0
        initializer_range=0.02, # 权重初始化时的标准差范围
        summary_type="cls_index",   # 下游任务中 summary 的类型（多用于分类）
        summary_use_proj=True,  # 是否对 summary 输出做线性变换
        summary_activation=None, # summary 的激活函数
        summary_proj_to_labels=True,   # 是否投影到标签维度（用于分类）
        summary_first_dropout=0.1,  # summary dropout 概率
        scale_attn_weights=True,   # 是否对 attention 权重进行缩放（默认 True）
        use_cache=True,  # 是否启用缓存，用于加速生成任务
        bos_token_id=50256,  # 起始 token ID（Beginning of Sequence）
        eos_token_id=50256,
        scale_attn_by_inverse_layer_idx=False, # 是否按层数反比缩放注意力（通常用于一些改进变种）
        reorder_and_upcast_attn=False,  # 是否在 attention 中重排序并上提精度（float32 计算）
        **kwargs,  # 其他额外参数
    ):
        self.vocab_size = vocab_size
        self.n_positions = n_positions
        self.n_embd = n_embd
        self.n_layer = n_layer
        self.n_head = n_head
        self.n_inner = n_inner
        self.activation_function = activation_function
        self.resid_pdrop = resid_pdrop
        self.embd_pdrop = embd_pdrop
        self.attn_pdrop = attn_pdrop
        self.layer_norm_epsilon = layer_norm_epsilon
        self.initializer_range = initializer_range
        self.summary_type = summary_type
        self.summary_use_proj = summary_use_proj
        self.summary_activation = summary_activation
        self.summary_first_dropout = summary_first_dropout
        self.summary_proj_to_labels = summary_proj_to_labels
        self.scale_attn_weights = scale_attn_weights
        self.use_cache = use_cache
        self.scale_attn_by_inverse_layer_idx = scale_attn_by_inverse_layer_idx
        self.reorder_and_upcast_attn = reorder_and_upcast_attn
        self.bos_token_id = bos_token_id
        self.eos_token_id = eos_token_id
        super().__init__(bos_token_id=bos_token_id, eos_token_id=eos_token_id, **kwargs) # 调用父类的初始化

In [9]:
configuration = GPT2Config()

In [10]:
configuration

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.51.3",
  "use_cache": true,
  "vocab_size": 50257
}

In [23]:
class GPT2OnnxConfig(OnnxConfigWithPast):
    def __init__(
        self,
        config: PretrainedConfig,
        task: str = "default",
        patching_specs: List[PatchingSpec] = None,
        use_past: bool = False,
    ):
        super().__init__(config, task=task, patching_specs=patching_specs, use_past=use_past)
        # 设置默认的填充id
        if not getattr(self._config, "pad_token_id", None): 
            # TODO: how to do that better?
            self._config.pad_token_id = 0
    # 这种装饰器叫做 @property 属性装饰器，它将一个方法转化为一个只读属性
    # 这个 inputs() 方法返回的是 ONNX 导出时模型输入的结构定义，也叫做 输入张量的动态维度映射说明，主要用于告诉 ONNX：
    # 每个输入的维度含义，比如哪个维度是 batch，哪个是 sequence，哪个是缓存长度（past_sequence）等。
    @property 
    def inputs(self) -> Mapping[str, Mapping[int, str]]:
        common_inputs = OrderedDict({"input_ids": {0: "batch", 1: "sequence"}})
        if self.use_past: # 如果使用缓存
            self.fill_with_past_key_values_(common_inputs, direction="inputs")
            common_inputs["attention_mask"] = {0: "batch", 1: "past_sequence + sequence"}
        else:
            common_inputs["attention_mask"] = {0: "batch", 1: "sequence"}

        return common_inputs

    @property 
    def num_layers(self) -> int:
        return self._config.n_layer

    @property
    def num_attention_heads(self) -> int:
        return self._config.n_head

    def generate_dummy_inputs(
        self,
        tokenizer: PreTrainedTokenizer,
        batch_size: int = -1,
        seq_length: int = -1,
        is_pair: bool = False,
        framework: Optional[TensorType] = None,
    ) -> Mapping[str, Any]:
        # 调用父类生成基本输入（input_ids 和 attention_mask），即正常推理用的输入。
        common_inputs = super(OnnxConfigWithPast, self).generate_dummy_inputs(
            tokenizer, batch_size=batch_size, seq_length=seq_length, is_pair=is_pair, framework=framework
        )
        # 按 forward() 函数参数顺序构建输入字典，从 input_ids 开始。
        ordered_inputs = OrderedDict({"input_ids": common_inputs["input_ids"]})
        # 如果启用缓存（KV缓存），则需要额外添加 past_key_values 输入。
        if self.use_past:
            if not is_torch_available():
                raise ValueError("Cannot generate dummy past_keys inputs without PyTorch installed.")
            else:
                import torch
                batch, seqlen = common_inputs["input_ids"].shape # 获取 batch 和当前序列长度
                # 设置 past 序列长度比当前序列长，模拟真实缓存场景
                past_key_values_length = seqlen + 2
                past_shape = ( # (b,h,s,hd)
                    batch,
                    self.num_attention_heads,
                    past_key_values_length,
                    self._config.hidden_size // self.num_attention_heads,
                )
                # 构造每层 KV 缓存的形状：[batch, num_heads, past_seq_len, head_dim]
                ordered_inputs["past_key_values"] = [
                    (torch.zeros(past_shape), torch.zeros(past_shape)) for _ in range(self.num_layers)
                ]
        # 添加当前序列的 attention mask。
        ordered_inputs["attention_mask"] = common_inputs["attention_mask"]
        # 如果使用 past，需将 attention_mask 补齐为 [batch, past_seq_len + cur_seq_len]，前面是 1 表示已有上下文。
        if self.use_past:
            mask_dtype = ordered_inputs["attention_mask"].dtype
            ordered_inputs["attention_mask"] = torch.cat(
                [ordered_inputs["attention_mask"], torch.ones(batch, past_key_values_length, dtype=mask_dtype)], dim=1
            )
        # 返回构建好的 dummy 输入，包括 input_ids、attention_mask 和 past_key_values（如果启用）。
        return ordered_inputs
    # 指定默认导出 ONNX 模型时使用的 ONNX opset 版本
    # default_onnx_opset = 13 是 GPT2 导出为 ONNX 时推荐使用的操作符版本号，确保兼容性和功能完整性。
    @property
    def default_onnx_opset(self) -> int:
        return 13
__all__ = ["GPT2Config", "GPT2OnnxConfig"]

In [11]:
onnxConfig=GPT2OnnxConfig(configuration)

In [15]:
onnxConfig.num_layers

12

In [16]:
onnxConfig.inputs

OrderedDict([('input_ids', {0: 'batch', 1: 'sequence'}),
             ('attention_mask', {0: 'batch', 1: 'sequence'})])

In [17]:
onnxConfig=GPT2OnnxConfig(configuration,use_past=True)

In [18]:
onnxConfig.inputs

OrderedDict([('input_ids', {0: 'batch', 1: 'sequence'}),
             ('past_key_values.0.key',
              {0: 'batch', 2: 'past_sequence + sequence'}),
             ('past_key_values.0.value',
              {0: 'batch', 2: 'past_sequence + sequence'}),
             ('past_key_values.1.key',
              {0: 'batch', 2: 'past_sequence + sequence'}),
             ('past_key_values.1.value',
              {0: 'batch', 2: 'past_sequence + sequence'}),
             ('past_key_values.2.key',
              {0: 'batch', 2: 'past_sequence + sequence'}),
             ('past_key_values.2.value',
              {0: 'batch', 2: 'past_sequence + sequence'}),
             ('past_key_values.3.key',
              {0: 'batch', 2: 'past_sequence + sequence'}),
             ('past_key_values.3.value',
              {0: 'batch', 2: 'past_sequence + sequence'}),
             ('past_key_values.4.key',
              {0: 'batch', 2: 'past_sequence + sequence'}),
             ('past_key_values.4.value',
   

In [19]:
onnxConfig.num_attention_heads

12

In [21]:
onnxConfig.default_onnx_opset

13

In [27]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('openai-community/gpt2')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [30]:
onnxConfig.generate_dummy_inputs(tokenizer,framework="pt").keys()

odict_keys(['input_ids', 'past_key_values', 'attention_mask'])

In [34]:
len([i for i in onnxConfig.generate_dummy_inputs(tokenizer,framework="pt").values()][1]) # past_key_values

12